In [ ]:
# Okay, great. We know what a climatology and anomaly is and how to calculate them. We calculated the anomaly at a location and identified the 90th percentile heat extreme events.
# Now that we have the dates of these events identified, let's take a look at how the meteorology of these events. We will do so by calculating composites of 
# various meteorological fields


# load in the necessary libraries
import scipy.io as scp
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import datetime as dt
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

# load in the hw dates file 
datesfile='/home/Veeshan.Narinesingh/INTERNS_2024/TUTORIALS/NYC_hwstardates_and_durations.nc'
hwdatesfilexr = xr.open_dataset(datesfile) 
startdates = hwdatesfilexr.event_startdates
durations = hwdatesfilexr.event_durations
numevents=len(durations)

In [ ]:
# load in the temperature too 
tdir = 'DATA/'
tfileprefix = 'ERA5_2deg_temperature.model_level_137.daily.'
ny1=1979 # specify the first year of the data files
ny2=2019 # specify the last year of the data files

# load in the temperature data for each year 
nyrs=ny2-ny1+1

time = np.zeros([365 * nyrs])
c1 = 0
c2 = 365

for yr in range(ny1,ny2 + 1):
    filenamet = tfileprefix + str(yr) + '.nc'
    ds = xr.open_dataset(tdir + filenamet)

    if yr == ny1:
        lat = ds.lat
        lon = ds.lon
        # preallocate t2m as an xarray
        var1np = np.zeros([365 * nyrs,np.size(lat),np.size(lon)])
        ratime = np.zeros([365 * nyrs])
        
    var1np[c1:c2,:,:] = ds.t[:365,:,:].values
    time[c1:c2] = np.arange(0,365,1) + 365 * (yr - ny1)
    ratime[c1:c2] = ds.time[:365]
    c1 = 1 * c2
    c2 = c2 + 365
var1 = xr.DataArray(data=var1np,dims=['time','lat','lon'],coords=dict(time=time, lat=lat, lon=lon))

In [ ]:
# let's also calculate the temperature anomaly

var1smooth5=var1.rolling(time=5,center=True,min_periods=1).mean()#5 day smooth using running mean
var1smooth5np=var1smooth5.values.reshape((nyrs,365,np.size(lat),np.size(lon)))
var1smooth5rs= xr.DataArray(data=var1smooth5np,dims=["year","day","lat","lon"],coords=dict(year=np.arange(ny1,ny2+1,1),day=np.arange(1,366,1)))
var1smooth5rs15day1=var1smooth5rs.rolling(day=15,center=True,min_periods=1).mean()#1st 15 day running mean 
var1smooth5rs15day2=var1smooth5rs15day1.rolling(day=15,center=True,min_periods=1).mean()#2nd 15 day running mean
clim=var1smooth5rs15day2.rolling(year=11,center=True,min_periods=1).mean()#29 day by 11 year climatology
anom=var1smooth5rs-clim
# reshape the array back to days,lat,lon from years,days,lat,lon
anomrs= anom.values.reshape(nyrs * 365, np.size(lat),np.size(lon))

In [ ]:
# let's plot the duration histogram again

#Calculate the average heat extreme duration and make a histogram
print('Average Duration: ' + str(durations.mean()) + ' days')

plt.hist(durations,bins=list(range(1, 16))) #bins=list(range(1, 16)) produces histogram bins that go to 1 to 15
plt.xlabel('Duration (days)')
plt.ylabel('Count')

In [ ]:
# Going forward let's just focus on the longest lasting events by picking the top half of the data

tophowmany=numevents // 2 # this divides a number by two and rounds down


# get indices for top events only
durationssortidx=np.argsort(-durations)
durationssort=durations[durationssortidx]


print('Average Duration: ' + str(np.mean(durations).values) + ' days')

print('Duration Standard Deviation: ' + str(np.std(durations).values) + ' days')


print('Average Duration of top ' + str(tophowmany) + ': ' + str(np.mean(durationssort[0:tophowmany]).values) + ' days')

print('Duration Standard Deviation of top ' + str(tophowmany) + ': ' + str(np.std(durationssort[0:tophowmany]).values) + ' days')

In [ ]:
# next find the indices of the start dates of just the top events

startdatesxr=xr.DataArray(data=startdates)
startdatesxrwant=startdatesxr[durationssortidx[0:tophowmany]]
    
indices = [0] * startdatesxrwant # preallocate 
for i in range(0,np.size(startdatesxrwant)):
    indices.values[i] = np.where(time == startdatesxrwant.values[i])[0]
    
indices

In [ ]:
# Excellent, so now we know the start dates of the top 48 longest events. On average, these events lasted around 6 days (but some were much longer!)


# Let's take a look at what the temperature anomaly looks like by grabbing the temperature anomalies on the start dates and averaging them all together and plotting
# this is called a composite of temperature anomaly over the starting dates of all events
tcomp = anomrs[indices,:,:].mean(axis=0)


# set lat and lon axis limits
lonwant=360-74 # NY is 74 West longitude, but the longitude dimension goes from 0 to 360, so we do 360-74
latwant=40.71
plotlims=[lonwant-60,lonwant+60,latwant-25,latwant+25]


# set temperature contours
tmax=5
tmin=-tmax
numcolors=10
tempcontoursf=np.linspace(tmin,tmax,numcolors)

counter=1
fig=plt.figure(figsize=(25,10))
#plot  composite 
ax = fig.add_subplot(1, 1, counter, projection=ccrs.EckertIII()) #1,1,1 a 1x1 subplot and the first entry
ax.set_global()
ax.coastlines('110m', alpha=0.9) #alpha is how dark lines are
plotto=ax.contourf(lon,lat,tcomp,transform=ccrs.PlateCarree(),cmap='bwr',levels=tempcontoursf)
fig.colorbar(plotto)
ax.set_extent(plotlims,crs=ccrs.PlateCarree())
plt.title("Temperature Anomaly (K), Day Relative to Start: " + str(0))
counter=counter+1


In [ ]:
# we can even see what it was like two days before by subtracting 2 from the starting indices


# Let's take a look at what the temperature anomaly looks like by grabbing the temperature anomalies on the start dates and averaging them all together and plotting
tcomp = anomrs[indices-2,:,:].mean(axis=0)


# set lat and lon axis limits
lonwant=360-74 # NY is 74 West longitude, but the longitude dimension goes from 0 to 360, so we do 360-74
latwant=40.71
plotlims=[lonwant-60,lonwant+60,latwant-25,latwant+25]


# set temperature contours
tmax=5
tmin=-tmax
numcolors=10
tempcontoursf=np.linspace(tmin,tmax,numcolors)

counter=1
fig=plt.figure(figsize=(25,10))
#plot  composite 
ax = fig.add_subplot(1, 1, counter, projection=ccrs.EckertIII()) #1,1,1 a 1x1 subplot and the first entry
ax.set_global()
ax.coastlines('110m', alpha=0.9) #alpha is how dark lines are
plotto=ax.contourf(lon,lat,tcomp,transform=ccrs.PlateCarree(),cmap='bwr',levels=tempcontoursf)
fig.colorbar(plotto)
ax.set_extent(plotlims,crs=ccrs.PlateCarree())
plt.title("Temperature Anomaly (K), Day Relative to Start: " + str(0))
counter=counter+1


In [ ]:
# 1. Make a plot for 4 days before events. Is there a clear signal?
# 2. If there is a signal for 4 days before, plot some days before that until you don't see a signal. 
# 3. Make a plot for 2 days after the start of events. The do 4, 6, and 8 days after
# 4. Describe how the events evolve  

In [ ]:
# Make sure you do the prompts above. Below is a code that will loop through and subplot the evolution of the temperature anomaly over time. You can use it to check your findings above.

counter=1 #start a figure counter 
fig=plt.figure(figsize=(25,10)) #specify the figure size
for t in range(-7,8): # start a loop that will go from -7 days before the start dates to +7 after

    # composite relative to the start date, 0 is on the start dates, -1 is 1 day before +1 is 1 day after
    comp = anomrs[indices + t,:,:].mean(axis=0)


    #plot  composite 
    ax = fig.add_subplot(3, 5, counter, projection=ccrs.EckertIII()) # specifying a subplot with 3 rows and 5 columns
    ax.set_global()
    ax.coastlines('110m', alpha=0.9) #alpha is how dark lines are
    plotto=ax.contourf(lon,lat,comp,transform=ccrs.PlateCarree(),cmap='bwr',levels=tempcontoursf)
    #fig.colorbar(plotto)
    ax.set_extent(plotlims,crs=ccrs.PlateCarree())
    plt.title("Day Relative to Start: " + str(t))
    counter=counter+1 # increase the fig counter by 1 each time you go through the loop
    
fig.colorbar(plotto)


In [ ]:
# Describe how the temperature evolves from the figure above. 

In [ ]:
# So we now have an idea of how the temperature evolves during these events. There are clear patterns and now we want a better sense of the dynamics at play.
# Next, let's take a look at how the 500 hPa geopotential height anomaly evolves during these events

# First load in the data
directoryz = 'DATA/'


c1 = 0
c2 = 365

for yr in range(ny1,ny2 + 1):
    filenamez = 'era5.z500.daily.2deg.' + str(yr) + '.nc'
    ds = xr.open_dataset(directoryz + filenamez)
    if yr == ny1:
        zlat = ds.lat
        zlon = ds.lon
        varz = np.zeros([365 * nyrs,np.size(zlat),np.size(zlon)])
    varz[c1:c2,:,:] = ds.z[:365,:,:].values
    c1 = 1 * c2
    c2 = c2 + 365
    
varz = xr.DataArray(data=varz,dims=['time','lat','lon'],coords=dict(time=time, lat=zlat, lon=zlon))

varzsmooth5=varz.rolling(time=5,center=True,min_periods=1).mean()#5 day smooth using running mean
varzsmooth5np=varzsmooth5.values.reshape((nyrs,365,np.size(zlat),np.size(zlon)))
varzsmooth5rs= xr.DataArray(data=varzsmooth5np,dims=["year","day","lat","lon"],coords=dict(year=np.arange(ny1,ny2+1,1),day=np.arange(1,366,1)))
varzsmooth5rs15day1=varzsmooth5rs.rolling(day=15,center=True,min_periods=1).mean()#1st 15 day running mean 
varzsmooth5rs15day2=varzsmooth5rs15day1.rolling(day=15,center=True,min_periods=1).mean()#2nd 15 day running mean
climz=varzsmooth5rs15day2.rolling(year=11,center=True,min_periods=1).mean()#29 day by 11 year climatology
anomz=varzsmooth5rs-climz

# reshape the array back to days,lat,lon from years,days,lat,lon
anomrsz= anomz.values.reshape(nyrs * 365, np.size(zlat),np.size(zlon))
climrsz=climz.values.reshape(nyrs * 365, np.size(zlat),np.size(zlon))

zlon=varz.lon.values
zlat=varz.lat.values

In [ ]:
# To get a sense of what the geopotential height looks like, let's plot the climatology of the 500 hPa Geopotential height, Z500.
# Midlatitude winds in the free atmosphere can be approximated as geostrophic, meaning the Coriolis Force is balanced by the Pressure Gradient Force
# an implication of this is that the winds in the atmosphere follow along lines of Geopotential Height

compraw = climrsz[indices,:,:].mean(axis=0)/9.8 # divide by 9.8, the acceleration due to gravity to go from units of geopotential to geopotential height in meters

zgrawlvl=np.arange(4500,6500,20)
fig=plt.figure(figsize=(15,10))

#plot  composite 
ax = fig.add_subplot(1, 1, 1, projection=ccrs.EckertIII()) #1,1,1 a 1x1 subplot and the first entry
ax.set_global()
ax.coastlines('110m', alpha=0.9) #alpha is how dark lines are
plottoraw=ax.contour(zlon,zlat,compraw,transform=ccrs.PlateCarree(),levels=zgrawlvl,linestyles='solid',colors='green')
ax.clabel(plottoraw, inline=True, fontsize=10)

ax.set_extent([-180,0,10,70],crs=ccrs.PlateCarree())
plt.title('Climatological Z500')

In [ ]:
# Now let's plot the composite of Z500 during the start day of the longest heat extreme events

compraw = varz[indices,:,:].mean(axis=0)/9.8 # divide by 9.8, the acceleration due to gravity to go from units of geopotential to geopotential height in meters

zgrawlvl=np.arange(4500,6500,20)
fig=plt.figure(figsize=(15,10))

#plot  composite 
ax = fig.add_subplot(1, 1, 1, projection=ccrs.EckertIII()) #1,1,1 a 1x1 subplot and the first entry
ax.set_global()
ax.coastlines('110m', alpha=0.9) #alpha is how dark lines are
plottoraw=ax.contour(zlon,zlat,compraw,transform=ccrs.PlateCarree(),levels=zgrawlvl,linestyles='solid',colors='green')
ax.clabel(plottoraw, inline=True, fontsize=10)
ax.set_extent([-180,0,10,70],crs=ccrs.PlateCarree())
plt.title('Climatological Z500')

In [ ]:
# How do the geopotential height contours in the climatology compare to the composite during the start of NYC heat events?

In [ ]:
#Let's add the raw the raw geopotential height field as well 

counter=1

fig=plt.figure(figsize=(25,10))

climitt=5
fig=plt.figure(figsize=(25,5))
for t in range(-7,8):

    # composite relative to the start date, 0 is on the start dates, -1 is 1 day before +1 is 1 day after
    compraw = varz[indices + t,:,:].mean(axis=0)/9.8

    
    compt = anomrs[indices + t,:,:].mean(axis=0)


    #plot  composite 
    ax = fig.add_subplot(3, 5, counter, projection=ccrs.EckertIII())
    ax.set_global()
    ax.coastlines('110m', alpha=0.9) #alpha is how dark lines are
    plottoraw=ax.contour(zlon,zlat,compraw,transform=ccrs.PlateCarree(),levels=zgrawlvl,linestyles='solid',colors='green')

    plotto2=ax.contourf(lon,lat,compt,transform=ccrs.PlateCarree(),cmap='bwr',levels=tempcontoursf)
    ax.set_extent([lonwant-90,lonwant+90,20,70],crs=ccrs.PlateCarree())
    plt.title("Day Relative to Start: " + str(t))
    counter=counter+1

fig.colorbar(plotto2)



In [ ]:
# How do the geopotential height contours evolve over time? What is the temperature doing as time progresses 

In [ ]:
# It is also useful to plot the geopotential height anomalies. This helps us determine the location of high and low pressure systems. Where there
# is a high pressure system, there is a positive geopotential height anomaly. Where there is a low pressure, there is a negative anomaly. 

#Here's code to get composites of geopotential height from 7 days before to 7 days after
counter=1
zglvlspos=np.arange(10,150,10)
zglvlsneg=np.arange(-150,-10,10)

fig=plt.figure(figsize=(25,5))
for t in range(-7,8):

    # composite relative to the start date, 0 is on the start dates, -1 is 1 day before +1 is 1 day after
    comp = anomrsz[indices + t,:,:].mean(axis=0)/9.8
    compt = anomrs[indices + t,:,:].mean(axis=0)


    #plot  composite 
    ax = fig.add_subplot(3, 5, counter, projection=ccrs.EckertIII()) #1,1,1 a 1x1 subplot and the first entry
    ax.set_global()
    ax.coastlines('110m', alpha=0.9) #alpha is how dark lines are
    plottopos=ax.contour(zlon,zlat,comp,transform=ccrs.PlateCarree(),levels=zglvlspos,linestyles='solid',colors='black')
    plottoneg=ax.contour(zlon,zlat,comp,transform=ccrs.PlateCarree(),levels=zglvlsneg,linestyles='dashed',colors='black')
    plotto2=ax.contourf(lon,lat,compt,transform=ccrs.PlateCarree(),cmap='bwr',levels=tempcontoursf)
    ax.set_extent([lonwant-90,lonwant+90,20,70],crs=ccrs.PlateCarree())
    plt.title("Day Relative to Start: " + str(t))
    counter=counter+1
fig.colorbar(plotto2)


In [ ]:
# How do the geopotential height anomaly contours evolve over time? What is the temperature doing as time progresses 